In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [3]:
import matplotlib.pyplot as plt

In [4]:
def train_classifier(model, lines, output, batches, iterations=1):
    errors = []
    lossfunc = nn.BCELoss()
    optimizer = optim.Adagrad(model.parameters())
    for _ in range(iterations):
        for batch in batches:
            try:
                if len(batch) == 20:
                    input = torch.cuda.LongTensor([lines[n] for n in batch])
                    truth = torch.cuda.FloatTensor([output[n] for n in batch])
                    truth[truth == 2] = 0
                    pred = model(input)
                    model.zero_grad()
                    loss = lossfunc(pred, truth)
                    loss.backward()
                    optimizer.step()
                    errors.append(loss.data)
            except:
                print('ad')
            print(batches.index(batch), end='\r')
            plt.plot(errors)

In [5]:
def get_summaries(model, lines_, batches, doc_folder, write_folder):
    for batch in batches:
        input = torch.cuda.LongTensor([lines_[n] for n in batch])
        pred = model(input)
        _idx = torch.sort(pred)[1].data
        for lines, docid in zip(_idx, batch):
            with open(doc_folder + str(docid)) as f:
                content = f.readlines()
                selected_lines = [content[l] for l in lines[0:3]]
                with open(write_folder + str(docid), 'w+') as f2:
                    f2.write(str(lines[0:3]) + '\n')
                    [f2.write(line) for line in selected_lines]

In [ ]:
def get_summary(doc_id, write_folder):
    print(open(write_folder + str(doc_id)).read())